# Data cleaning

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
# Importamos el CSV
input_path = Path("data") / "staySpain_raw.csv"
df = pd.read_csv(input_path)

In [3]:
# Hacemos una copia de df para no modificar el df original
df_clean = df.copy()

In [4]:
# Info de df_clean
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   apartment_id                 8000 non-null   int64  
 1   name                         7997 non-null   object 
 2   description                  7946 non-null   object 
 3   host_id                      8000 non-null   int64  
 4   neighbourhood_name           8000 non-null   object 
 5   neighbourhood_district       4861 non-null   object 
 6   room_type                    8000 non-null   object 
 7   accommodates                 8000 non-null   int64  
 8   bathrooms                    7957 non-null   float64
 9   bedrooms                     7961 non-null   float64
 10  beds                         7992 non-null   float64
 11  amenities_list               7983 non-null   object 
 12  price                        7829 non-null   float64
 13  minimum_nights    

In [5]:
# Visualizamos todas las columnas en el output
pd.set_option("display.max_columns", None)

In [6]:
# Comprobamos si hay filas enteras duplicadas
df_clean.duplicated().any()

False

In [7]:
# Comprobamos si hay duplicados en "apartment_id"
df_clean.duplicated(subset="apartment_id").sum()

307

In [8]:
# Buscamos duplicados en "apartment_id"
duplicados = df_clean[df_clean.duplicated("apartment_id", keep=False)]

In [9]:
# Ordenamos los registros para eliminar los duplicados con "insert_date" más antigua
df_clean = df_clean.sort_values("insert_date", ascending=False)\
                     .drop_duplicates("apartment_id", keep='first')\
                     .sort_index()

In [10]:
import re, unicodedata

def clean_string(input_string):
    s = "" if input_string is None else str(input_string)

    # Normaliza Unicode (arregla acentos rotos y el carácter �)
    s = unicodedata.normalize("NFKC", s).replace("�", "")

    # Conserva letras latinas (con tildes), números, espacios, apóstrofes y guiones
    s = re.sub(r"[^0-9A-Za-zÀ-ÖØ-öø-ÿ' -]", " ", s)

    # Colapsa espacios y recorta extremos
    s = re.sub(r"\s+", " ", s).strip()

    # Pone mayúscula la primera letra de CADA palabra (mejor para nombres propios)
    return s.title()


In [11]:
# Aplicamos la función de limpieza a las columnas que se usan en la imputación
cols_to_clean = ["room_type", "neighbourhood_name"]
for col in cols_to_clean:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].apply(clean_string)


In [12]:
df_clean.dtypes

apartment_id                     int64
name                            object
description                     object
host_id                          int64
neighbourhood_name              object
neighbourhood_district          object
room_type                       object
accommodates                     int64
bathrooms                      float64
bedrooms                       float64
beds                           float64
amenities_list                  object
price                          float64
minimum_nights                   int64
maximum_nights                   int64
has_availability                object
availability_30                  int64
availability_60                  int64
availability_90                  int64
availability_365                 int64
number_of_reviews                int64
first_review_date               object
last_review_date                object
review_scores_rating           float64
review_scores_accuracy         float64
review_scores_cleanliness

In [13]:
# Conversión de datos a date en las columnas que muestran fechas
df_clean["first_review_date"] = pd.to_datetime(df_clean['first_review_date'], errors='coerce', format='%d/%m/%Y')
df_clean["last_review_date"] = pd.to_datetime(df_clean['last_review_date'], errors='coerce', format='%d/%m/%Y')
df_clean["insert_date"] = pd.to_datetime(df_clean['insert_date'], errors='coerce', format='%d/%m/%Y')

In [14]:
# Conversión de float a int
df_clean["bathrooms"] = df_clean['bathrooms'].astype("Int64")
df_clean["bedrooms"] = df_clean['bedrooms'].astype("Int64")
df_clean["beds"] = df_clean['beds'].astype("Int64")

In [15]:
# Creamos una columna nueva para convertir "is_instant_bookable" en booleano
df_clean["is_instant_bookable_bool"] = df_clean["is_instant_bookable"].replace({"VERDADERO": True, "FALSO": False}).astype(bool)

df_clean.head()

/tmp/ipykernel_4879/662236793.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean["is_instant_bookable_bool"] = df_clean["is_instant_bookable"].replace({"VERDADERO": True, "FALSO": False}).astype(bool)


,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,beds,amenities_list,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review_date,last_review_date,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,is_instant_bookable,reviews_per_month,country,city,insert_date,is_instant_bookable_bool
0,11964,A ROOM WITH A VIEW,Private bedroom in our attic apartment. Right ...,45553,Centro,NaN,Private Room,2,2,1,1,"TV,Internet,Wifi,Air conditioning,Elevator,Buz...",400.0,3,365,VERDADERO,7,20,40,130,78,2010-01-02,2017-09-05,970.0,100.0,100.0,100.0,100.0,100.0,100.0,FALSO,75.0,spain,malaga,2018-07-31,False
1,21853,Bright and airy room,We have a quiet and sunny room with a good vie...,83531,Crmenes,Latina,Private Room,1,1,1,1,"TV,Internet,Wifi,Air conditioning,Kitchen,Free...",170.0,4,40,VERDADERO,0,0,0,162,33,2014-10-10,2018-07-15,920.0,90.0,90.0,100.0,100.0,80.0,90.0,FALSO,52.0,spain,madrid,2020-01-10,False
2,32347,Explore Cultural Sights from a Family-Friendly...,Open French doors and step onto a plant-filled...,139939,San Vicente,Casco Antiguo,Entire Home Apt,4,1,2,2,"TV,Internet,Wifi,Air conditioning,Wheelchair a...",990.0,2,120,VERDADERO,26,31,31,270,148,2011-01-05,2019-07-22,980.0,100.0,100.0,100.0,100.0,100.0,100.0,VERDADERO,142.0,spain,sevilla,2019-07-29,True
3,35379,Double 02 CasanovaRooms Barcelona,Room at a my apartment. Kitchen and 2 bathroom...,152232,L'Antiga Esquerra De L'Eixample,Eixample,Private Room,2,2,1,1,"TV,Internet,Wifi,Kitchen,Breakfast,Elevator,Bu...",400.0,2,730,VERDADERO,9,23,49,300,292,2012-03-13,2020-01-04,940.0,100.0,90.0,100.0,100.0,100.0,90.0,VERDADERO,306.0,spain,barcelona,2020-01-10,True
4,35801,Can Torras Farmhouse Studio Suite,Lay in bed & watch sunlight change the mood of...,153805,Quart,NaN,Private Room,5,1,2,5,"Wifi,Pool,Free parking on premises,Breakfast,P...",900.0,1,180,VERDADERO,0,19,49,312,36,2011-07-08,2018-08-08,970.0,100.0,100.0,100.0,100.0,100.0,100.0,FALSO,39.0,spain,girona,2019-02-19,False


In [16]:
# Eliminamos la columna original
df_clean = df_clean.drop(columns="is_instant_bookable")

# Renombramos la nueva columna
df_clean = df_clean.rename({"is_instant_bookable_bool":"is_instant_bookable"}, axis=1)

df_clean.head()

,apartment_id,name,description,host_id,neighbourhood_name,neighbourhood_district,room_type,accommodates,bathrooms,bedrooms,beds,amenities_list,price,minimum_nights,maximum_nights,has_availability,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review_date,last_review_date,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,reviews_per_month,country,city,insert_date,is_instant_bookable
0,11964,A ROOM WITH A VIEW,Private bedroom in our attic apartment. Right ...,45553,Centro,NaN,Private Room,2,2,1,1,"TV,Internet,Wifi,Air conditioning,Elevator,Buz...",400.0,3,365,VERDADERO,7,20,40,130,78,2010-01-02,2017-09-05,970.0,100.0,100.0,100.0,100.0,100.0,100.0,75.0,spain,malaga,2018-07-31,False
1,21853,Bright and airy room,We have a quiet and sunny room with a good vie...,83531,Crmenes,Latina,Private Room,1,1,1,1,"TV,Internet,Wifi,Air conditioning,Kitchen,Free...",170.0,4,40,VERDADERO,0,0,0,162,33,2014-10-10,2018-07-15,920.0,90.0,90.0,100.0,100.0,80.0,90.0,52.0,spain,madrid,2020-01-10,False
2,32347,Explore Cultural Sights from a Family-Friendly...,Open French doors and step onto a plant-filled...,139939,San Vicente,Casco Antiguo,Entire Home Apt,4,1,2,2,"TV,Internet,Wifi,Air conditioning,Wheelchair a...",990.0,2,120,VERDADERO,26,31,31,270,148,2011-01-05,2019-07-22,980.0,100.0,100.0,100.0,100.0,100.0,100.0,142.0,spain,sevilla,2019-07-29,True
3,35379,Double 02 CasanovaRooms Barcelona,Room at a my apartment. Kitchen and 2 bathroom...,152232,L'Antiga Esquerra De L'Eixample,Eixample,Private Room,2,2,1,1,"TV,Internet,Wifi,Kitchen,Breakfast,Elevator,Bu...",400.0,2,730,VERDADERO,9,23,49,300,292,2012-03-13,2020-01-04,940.0,100.0,90.0,100.0,100.0,100.0,90.0,306.0,spain,barcelona,2020-01-10,True
4,35801,Can Torras Farmhouse Studio Suite,Lay in bed & watch sunlight change the mood of...,153805,Quart,NaN,Private Room,5,1,2,5,"Wifi,Pool,Free parking on premises,Breakfast,P...",900.0,1,180,VERDADERO,0,19,49,312,36,2011-07-08,2018-08-08,970.0,100.0,100.0,100.0,100.0,100.0,100.0,39.0,spain,girona,2019-02-19,False


In [17]:
# # Repetimos proceso con has_availability pero con map() para mantener los NaN
mapping={"VERDADERO":True, "FALSO":False}
df_clean["has_availability"] = df_clean["has_availability"].map(mapping)


In [18]:
df_clean["has_availability"].isna().sum()

534

In [19]:
# Imputación de nombre para registros con NaN en "name"
df_clean['name'] = df_clean['name'].fillna(df_clean['room_type'] + ' ' + df_clean['neighbourhood_name'])

In [20]:
# Imputación de descripción para registros con NaN en "description"
df_clean['description'] = df_clean['description'].fillna(df_clean['name'])

# last part

In [21]:
"""Formatear las variables de review"""
    
df_clean['review_scores_rating'] = df_clean['review_scores_rating'] / 10
df_clean['review_scores_accuracy'] = df_clean['review_scores_accuracy'] / 10
df_clean['review_scores_cleanliness'] = df_clean['review_scores_cleanliness'] / 10
df_clean['review_scores_checkin'] = df_clean['review_scores_checkin'] / 10
df_clean['review_scores_communication'] = df_clean['review_scores_communication'] / 10
df_clean['review_scores_location'] = df_clean['review_scores_location'] / 10
df_clean['review_scores_value'] = df_clean['review_scores_value'] / 10


In [22]:
def input_price(df_clean, price, room_type, accommodates, neighbourhood_name):
    """Imputación de precios"""
    df = df_clean.copy()  # Copia del DataFrame original
    avgs = df.groupby([room_type, accommodates, neighbourhood_name])[price].transform('mean')
    df[price] = df[price].fillna(avgs)
    return df

# Aplicamos la función y guardamos el resultado en df_clean para conservar las transformaciones anteriores
df_clean = input_price(df_clean, 'price', 'room_type', 'accommodates', 'neighbourhood_name')


In [23]:
# Ver las filas donde 'price' es nulo luego de aplicar la formula de inputacion
cols = [c for c in ['id', 'room_type', 'accommodates', 'neighbourhood_name', 'price'] if c in df_clean.columns]
filas_nulas = df_clean[df_clean['price'].isna()][cols]
print(f"Filas con 'price' nulo: {len(filas_nulas)}")
filas_nulas


Filas con 'price' nulo: 46


,room_type,accommodates,neighbourhood_name,price
85,Entire Home Apt,11,El Fort Pienc,NaN
476,Entire Home Apt,12,Escorca,NaN
780,Entire Home Apt,10,El Carme,NaN
929,Entire Home Apt,10,Binissalem,NaN
1161,Entire Home Apt,16,Vidr,NaN
1280,Entire Home Apt,12,Art,NaN
1394,Entire Home Apt,16,Consell,NaN
1650,Entire Home Apt,16,Lloseta,NaN
1678,Entire Home Apt,16,El Barri Gtic,NaN
2043,Entire Home Apt,16,Palma De Mallorca,NaN


In [24]:
# Segunda imputación para lo que no pudo completarse con la media grupal
df_clean['price'] = df_clean['price'].fillna(df_clean['price'].mean())

In [25]:

from pathlib import Path
import pandas as pd

# 1) Carpeta de salida (crea /data si no existe)
out_dir = Path("data")
out_dir.mkdir(parents=True, exist_ok=True)

# 2) Rutas de salida
pkl_path = out_dir / "staySpain_cleaned.pkl"
csv_path = out_dir / "staySpain_cleaned.csv"

# 3) Guardar el df LIMPIO
df_clean.to_pickle(pkl_path)
df_clean.to_csv(csv_path, index=False)

# 4) Verificación inmediata: recargar y comparar
df_check = pd.read_pickle(pkl_path)

print("Guardado en:")
print(" - PKL:", pkl_path.resolve())
print(" - CSV:", csv_path.resolve())
print("\nVerificación:")
print(" - Shapes iguales:", df_check.shape == df_clean.shape)
print(" - Columnas iguales:", list(df_check.columns) == list(df_clean.columns))
print(" - Tipos iguales:", (df_check.dtypes == df_clean.dtypes).all())



Guardado en:
 - PKL: /home/soliton/SYNCH_filgaia/ProjecteData/Equip_17/week_02_2025-09-22/data/staySpain_cleaned.pkl
 - CSV: /home/soliton/SYNCH_filgaia/ProjecteData/Equip_17/week_02_2025-09-22/data/staySpain_cleaned.csv

Verificación:
 - Shapes iguales: True
 - Columnas iguales: True
 - Tipos iguales: True
